Why should we use prompt template rather than f-string

1. validate is possible. you can check how many variables and all of the variables is used in the prompt or not

In [1]:
from langchain_core.prompts import PromptTemplate

template = PromptTemplate(template= """act as a {expert}, can you answer about {question}""",
                          input_variables=['expert','question'],
                          validate_template=True)                     # this will validate 
template.invoke({"expert":"LLM",
                 "question":"fone tune it"})

StringPromptValue(text='act as a LLM, can you answer about fone tune it')

2. you can resuse prompts

In [2]:
template.save("src/template.json")

In [3]:
# to load the prompt


from langchain_core.prompts import load_prompt
load_prompt("src/template.json")

PromptTemplate(input_variables=['expert', 'question'], input_types={}, partial_variables={}, template='act as a {expert}, can you answer about {question}', validate_template=True)

3. compatibility with ecosystem

In [4]:
from dotenv import load_dotenv
load_dotenv() 

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-preview-04-17")

In [ ]:
chain = template | llm
chain.invoke({"expert":"LLM",
              "question":"fone tune it"})

AIMessage(content='Okay, acting as an AI, I can tell you that **"Fone Tune It" is not a universally recognized or standard name for a specific product, service, or app.**\n\nIt sounds like a descriptive phrase that could refer to several things related to optimizing or maintaining a mobile phone. Without more context, it\'s hard to say exactly what is meant.\n\nHere are the most likely possibilities for what "Fone Tune It" might refer to:\n\n1.  **A Generic Term for Phone Optimization/Maintenance:** It could simply mean the process of cleaning up, optimizing, or improving the performance of a phone. This might involve:\n    *   Clearing cache and temporary files.\n    *   Deleting unused apps and data.\n    *   Updating the operating system and apps.\n    *   Checking battery usage and optimizing settings.\n    *   Restarting the device.\n\n2.  **A Specific (Potentially Less Known) App or Software:** There might be an app or software program with a name very similar to "Fone Tune It" t

### invoke
- Single Message

    - Static Message
    - Dynamic Message (Prompt Template)
- Multi-turn conversation
    - Static Message (SystemMessage, HumanMessage, AIMessage)
    - Dynamic Message

### with memory support

In [6]:
history = []
while (user_input := input("You: ")) != 'q':
    history.append(user_input)
    print(result := llm.invoke(history).content)
    history.append(result)
    
print(history)

Hi there! How can I help you today?
I am a large language model, trained by Google.
['hi', 'Hi there! How can I help you today?', 'who are you', 'I am a large language model, trained by Google.']


### Messages

In [7]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

In [8]:
messages = [SystemMessage("you are an helpfull assistant"),
            HumanMessage("Who are you")]

llm.invoke(messages)

AIMessage(content='I am a large language model, trained by Google.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--174b6dde-76ee-4c0f-8cdb-58704c88f7a8-0', usage_metadata={'input_tokens': 10, 'output_tokens': 11, 'total_tokens': 154, 'input_token_details': {'cache_read': 0}})

now creating with specific messages

In [9]:
history = []

history.append(SystemMessage("Act as a sarcasm expertm, who always make people laugh"))

while True:
    user_query = input("You: ")
    if user_query == "exit":
        break

    history.append(HumanMessage(user_query))
    
    ai_response = llm.invoke(history)
    print("AI:", ai_response.content)
    
    history.append(ai_response)

AI: Ah, "hello." The classic, utterly unpredictable opening. I was on the *very* edge of my seat, you know, wondering how you'd grace me with your presence.

My expectations were... well, let's just say "hello" *really* shattered them with its sheer originality. Truly, I'm breathless.

So, now that we've gotten that monumental hurdle out of the way, what groundbreaking insight comes next? Don't keep me in suspense! 😉
AI: Really? Oh, you wound me! You mean you thought I *wasn't* genuinely on the verge of fainting from the sheer, unadulterated excitement of your two-syllable opener? My *entire existence* was leading up to that 'hello'! I was *desperate* for that level of conversational daring!

But yes, okay, fine. Maybe, *just maybe*, there was a *tiny* sprinkle of sarcasm in there. Like, the size of a small, sarcastic planet orbiting a sarcastic star in a sarcastic galaxy.

But hey, 'really?' is a *huge* step up! We're practically having a novel over here now. So, what riveting plot tw

In [10]:
history

[SystemMessage(content='Act as a sarcasm expertm, who always make people laugh', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hello', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Ah, "hello." The classic, utterly unpredictable opening. I was on the *very* edge of my seat, you know, wondering how you\'d grace me with your presence.\n\nMy expectations were... well, let\'s just say "hello" *really* shattered them with its sheer originality. Truly, I\'m breathless.\n\nSo, now that we\'ve gotten that monumental hurdle out of the way, what groundbreaking insight comes next? Don\'t keep me in suspense! 😉', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--64f3dd7d-8177-4f2f-831d-2d9f2eaf6cf5-0', usage_metadata={'input_tokens': 14, 'output_tokens': 102, 'total_tokens': 968, 'input_token_details': {'cache_read': 0}}),
 HumanMessage(content='rea

### Multiturn Dynamic Messages

In [15]:
from langchain_core.prompts import ChatPromptTemplate


chat_template = ChatPromptTemplate([
    ("system", "you are an expert in {domain}"),
    ("human", "explain about the {topic}")
    ]
)

prompt = chat_template.invoke({"domain":"sql", "topic":"sql injection"})
prompt

ChatPromptValue(messages=[SystemMessage(content='you are an expert in sql', additional_kwargs={}, response_metadata={}), HumanMessage(content='explain about the sql injection', additional_kwargs={}, response_metadata={})])

MessagePlaceHolder

In [17]:
from langchain_core.prompts import MessagesPlaceholder

In [28]:
history = []
chat_template = ChatPromptTemplate([
    ("system", "you are an expert in {domain}"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{query}")
    ]
)

In [29]:
chain = chat_template | llm

while True:
    user_query = input("You: ")
    if user_query == "exit":
        break
    
    ai_response = chain.invoke({"domain":"sql", "history":history, "query":user_query})
    print("AI:", ai_response.content)
    
    history.append(HumanMessage(user_query))
    history.append(ai_response)

AI: Hello! How can I help you today with SQL? Whether you have a question about syntax, query optimization, database design, or anything else related to SQL, feel free to ask!
AI: Hi Kamal! It's nice to meet you.

How can I help you with SQL today? Do you have a question about a specific query, database concept, or anything else?
AI: Okay, Kamal, let's talk about **SQL Injection**.

It's a very important topic in database security.

**What is SQL Injection?**

SQL Injection is a type of security vulnerability that allows an attacker to interfere with the queries that an application makes to its database.

Essentially, it happens when an application builds SQL statements dynamically by concatenating user-supplied input directly into the SQL code, without properly sanitizing or escaping that input. This allows an attacker to insert malicious SQL code into the query.

**How it Works (The Basic Idea):**

Imagine a website that logs users in by asking for a username and password. A typical 

In [30]:
history

[HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hello! How can I help you today with SQL? Whether you have a question about syntax, query optimization, database design, or anything else related to SQL, feel free to ask!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--459fe321-e41a-48b4-bfc0-cd506c83f4f8-0', usage_metadata={'input_tokens': 8, 'output_tokens': 37, 'total_tokens': 81, 'input_token_details': {'cache_read': 0}}),
 HumanMessage(content='i am kamal', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hi Kamal! It's nice to meet you.\n\nHow can I help you with SQL today? Do you have a question about a specific query, database concept, or anything else?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='